In [1]:
import numpy as np
import pandas as pd
from ortools.graph import pywrapgraph
from collections import defaultdict

# Triplets

In [2]:
triplets = 5001
wish = pd.read_csv('Data/child_wishlist_v2.csv', header=None).as_matrix()[:triplets, 1:]
#sant = pd.read_csv('Data/gift_goodkids_v2.csv', header=None).as_matrix()[:, 1:]

In [3]:
edgeMap = defaultdict(int)
child3 = 0
for i in range(0,triplets,3):
    for j in range(3):
        child = i + j
        for gift in range(100):
            edgeMap[(child3, wish[child][gift])] += (100 - gift)*2
    child3 += 1

In [4]:
padding = 1667
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
supplies = []

for h in edgeMap:
    c, g = h
    start_nodes.append(int(c))
    end_nodes.append(int(padding + g))
    capacities.append(1)
    unit_costs.append(-edgeMap[h])

In [5]:
for i in range(padding):
    supplies.append(1)
for j in range(padding, padding+1000):
    supplies.append(-1000)

In [6]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [7]:
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], unit_costs[i])

In [8]:
# Add node supplies.
for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

In [9]:
# Find the minimum cost flow
print('Start solve....')
min_cost_flow.SolveMaxFlowWithMinCost()
res1 = min_cost_flow.MaximumFlow()
print('Maximum flow:', res1)
res2 = min_cost_flow.OptimalCost()
print('Optimal cost:', -res2 )
print('Num arcs:', min_cost_flow.NumArcs())

Start solve....
('Maximum flow:', 1667)
('Optimal cost:', 653360)
('Num arcs:', 438244)


In [10]:
pred = defaultdict(int)
for i in range(min_cost_flow.NumArcs()):
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    if cost != 0:
        pred[min_cost_flow.Tail(i)] = min_cost_flow.Head(i) - padding

In [11]:
happiness = np.zeros(1667, dtype=np.int32)
for child3 in range(1667):
    gift = pred[child3]
    child_happiness = 0
    for j in range(3):
        child = child3*3 + j
        flag = np.where(wish[child]==gift)[0] + 1
        if not flag:
            child_happiness += -1
        else:
            child_happiness += (101 - flag[0]) * 2
    happiness[child3] = child_happiness

In [12]:
unhappy = np.where(happiness < 350)[0]
print "Number of triplets unhappy", len(unhappy)
for i in unhappy:
    pred[i] = -1
    
answ3 = np.zeros(3*padding, dtype=np.int32)
answ3[:] = -1
for i in pred.keys():
    answ3[i*3] = pred[i]
    answ3[i*3 + 1] = pred[i]
    answ3[i*3 + 2] = pred[i]

Number of triplets unhappy 253


# Twins

In [13]:
twins = 45001
wish = pd.read_csv('Data/child_wishlist_v2.csv', header=None).as_matrix()[triplets:twins, 1:]

In [14]:
n_twins = 40000
edgeMap = defaultdict(int)
child2 = 0
for i in range(0,n_twins,2):
    for j in range(2):
        child = i + j
        for gift in range(100):
            edgeMap[(child2, wish[child][gift])] += (100 - gift)*2
    child2 += 1
    
print child2

20000


In [15]:
padding = 20000
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
supplies = []

for h in edgeMap:
    c, g = h
    start_nodes.append(int(c))
    end_nodes.append(int(padding + g))
    capacities.append(1)
    unit_costs.append(-edgeMap[h])

In [16]:
for i in range(padding):
    supplies.append(1)
for j in range(padding, padding+1000):
    supplies.append(-1000)

In [17]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [18]:
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], unit_costs[i])

In [19]:
# Add node supplies.
for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

In [20]:
# Find the minimum cost flow
print('Start solve....')
min_cost_flow.SolveMaxFlowWithMinCost()
res1 = min_cost_flow.MaximumFlow()
print('Maximum flow:', res1)
res2 = min_cost_flow.OptimalCost()
print('Optimal cost:', -res2 )
print('Num arcs:', min_cost_flow.NumArcs())

Start solve....
('Maximum flow:', 20000)
('Optimal cost:', 6691120)
('Num arcs:', 3739339)


In [21]:
pred = defaultdict(int)
for i in range(min_cost_flow.NumArcs()):
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    if cost != 0:
        pred[min_cost_flow.Tail(i)] = min_cost_flow.Head(i) - padding

In [22]:
happiness = np.zeros(20000, dtype=np.int32)
for child2 in range(20000):
    gift = pred[child2]
    child_happiness = 0
    for j in range(2):
        child = child2*2 + j
        flag = np.where(wish[child]==gift)[0] + 1
        if not flag:
            child_happiness += -1
        else:
            child_happiness += (101 - flag[0]) * 2
    happiness[child2] = child_happiness

In [26]:
unhappy = np.where(happiness < 260)[0]
print "Number of triplets unhappy", len(unhappy)
for i in unhappy:
    pred[i] = -1
    
answ2 = np.zeros(2*padding, dtype=np.int32)
answ2[:] = -1
for i in pred.keys():
     answ2[i*2] = pred[i]
     answ2[i*2 + 1] = pred[i]

Number of triplets unhappy 537


# combine triplets and twins

In [33]:
answ = np.zeros(twins, dtype=np.int32)
answ[:] = -1

In [34]:
for i in range(len(answ3)):
    answ[i] = answ3[i]
for i in range(len(answ2)):
    answ[i+triplets] = answ2[i]

In [35]:
from collections import Counter
counts = Counter(elem for elem in answ)

# Single

In [37]:
twins = 45001
wish = pd.read_csv('Data/child_wishlist_v2.csv', header=None).as_matrix()[twins:, 1:]
#sant = pd.read_csv('Data/gift_goodkids_v2.csv', header=None).as_matrix()[:, 1:]

In [38]:
edgeMap = defaultdict(int)
for child in range(wish.shape[0]):
    for gift in range(100):
        edgeMap[(child, wish[child][gift])] += (1 + (100 - gift)*2)*100

KeyboardInterrupt: 

In [27]:
# for gift in range(sant.shape[0]):
#     for i in range(sant.shape[1]):
#         child = sant[gift][i] - twins
#         if child >= 0 and child < 10000: #must modified
#             child2 = child / 2
#             edgeMap[(child2, gift)] += 1 + (1000 - i)*2

In [28]:
padding = wish.shape[0]
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
supplies = []

for h in edgeMap:
    c, g = h
    start_nodes.append(int(c))
    end_nodes.append(int(padding + g))
    capacities.append(1)
    unit_costs.append(-edgeMap[h])

In [29]:
for i in range(padding):
    supplies.append(1)
for j in range(padding, padding+1000):
    supplies.append(-1000+counts[j])

In [30]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [31]:
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], unit_costs[i])

In [32]:
# Add node supplies.
for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i-padding])

In [33]:
# Find the minimum cost flow
print('Start solve....')
min_cost_flow.SolveMaxFlowWithMinCost()
res1 = min_cost_flow.MaximumFlow()
print('Maximum flow:', res1)
res2 = min_cost_flow.OptimalCost()
print('Optimal cost:', -res2 )
print('Num arcs:', min_cost_flow.NumArcs())

Start solve....
('Maximum flow:', 10000)
('Optimal cost:', 201037445)
('Num arcs:', 1008836)


In [34]:
answ1 = np.zeros(padding, dtype=np.int32)
answ1[:] = -1
for i in range(min_cost_flow.NumArcs()):
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    if cost != 0:
        answ1[min_cost_flow.Tail(i)] = min_cost_flow.Head(i) - padding

# write out put

In [37]:
out = open('subm.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(answ)):
    out.write(str(i) + ',' + str(answ[i]) + '\n')
for i in range(len(answ1)):
    out.write(str(i+twins) + ',' + str(answ1[i]) + '\n')
out.close()